In [3]:
import os
import json
import numpy as np
from PIL import Image, ImageDraw
from torchvision import transforms
import random
import shutil


In [ ]:
json_dir = "data/sampled_rice_images/json"
output_dir = "data/sampled_rice_masks"
os.makedirs(output_dir, exist_ok=True)

for fname in os.listdir(json_dir):
    if not fname.endswith(".json"):
        continue

    path = os.path.join(json_dir, fname)
    with open(path, "r") as f:
        data = json.load(f)

    img_shape = data.get("imageHeight"), data.get("imageWidth")
    mask = Image.new("L", (img_shape[1], img_shape[0]), 0)  # (width, height)
    draw = ImageDraw.Draw(mask)

    for shape in data["shapes"]:
        points = shape["points"]
        polygon = [tuple(p) for p in points]
        draw.polygon(polygon, outline=255, fill=255)

    out_path = os.path.join(output_dir, fname.replace(".json", ".png"))
    mask.save(out_path)

In [4]:
import os
image_dir = "data/rice_images"           # Input images in subfolders
mask_dir = "data/generated_rice_masks"         # Corresponding masks in same subfolder structure
output_dir = "data/masked_rice_images"         # Output masked images
os.makedirs(output_dir, exist_ok=True)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

for root, _, files in os.walk(image_dir):
    for fname in files:
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        img_path = os.path.join(root, fname)
        relative_path = os.path.relpath(img_path, image_dir)
        mask_path = os.path.join(mask_dir, relative_path)
        out_path = os.path.join(output_dir, relative_path)

        if not os.path.exists(mask_path):
            print(f"Skipping {relative_path} — no mask found")
            continue

        try:
            os.makedirs(os.path.dirname(out_path), exist_ok=True)

            image = transform(Image.open(img_path).convert("RGB"))
            mask = transform(Image.open(mask_path).convert("L"))
            masked = image * mask  # broadcast

            masked_img = transforms.ToPILImage()(masked)
            masked_img.save(out_path)
        except Exception as e:
            print(f"[ERROR] Failed to process {img_path}: {e}")


In [ ]:

def sample_images_per_class(source_dir="data/rice_images", dest_dir="data/sampled_rice_images", num_samples=10):
    os.makedirs(dest_dir, exist_ok=True)

    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        image_files = [f for f in os.listdir(class_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        sampled_files = random.sample(image_files, min(num_samples, len(image_files)))

        dest_class_path = os.path.join(dest_dir, class_name)
        os.makedirs(dest_class_path, exist_ok=True)

        for file_name in sampled_files:
            src = os.path.join(class_path, file_name)
            dst = os.path.join(dest_class_path, file_name)
            shutil.copy2(src, dst)

        print(f"✅ Sampled {len(sampled_files)} images from {class_name}")

# Run it
sample_images_per_class("data/rice_images", "data/sampled_rice_images", num_samples=10)


✅ Sampled 10 images from blast
✅ Sampled 10 images from brown_spot
✅ Sampled 10 images from hispa
✅ Sampled 10 images from normal
